# List 1

## Imports

In [1]:
import polars as pl
from pathlib import Path
from collections import defaultdict
from pydantic import BaseModel, computed_field, ConfigDict
from itertools import chain
import logging
from pprint import pprint

In [2]:
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [3]:
class Paths(BaseModel):
    model_config = ConfigDict(frozen=True)
    project: Path = Path(".")

    @computed_field
    @property
    def data(self) -> Path:
        return self.project / "data"

    
PATHS = Paths()

## Load

In [4]:
def load_data() -> dict[str, pl.LazyFrame]:
    """Using lazy frame during load allow for comfortable handle huge csv.
    In that case those big files are not materialized on load (what crashes 
    my computer).
    """
    return {
        doc: pl.scan_csv(PATHS.data / doc, separator=sep)
        for doc, sep in [
            ("Branches_ENG.csv", ";"),
            ("Categories_ENG.csv", ";"),
            ("Customers_ENG.csv", ";"),
            ("Order_Details.csv", ","),
            ("Orders.csv", ","),
        ] 
    }

In [5]:
def get_features(frames: dict[str, pl.LazyFrame]) -> set[str]:
    return set(
        chain.from_iterable(frame.collect_schema().names()
        for frame in frames.values())
    )

In [6]:
frames = load_data()

In [7]:
pprint(get_features(frames))

{'ADDRESSTEXT',
 'AMOUNT',
 'BRANCH_ID',
 'BRANCH_TOWN',
 'BRAND',
 'CATEGORY1',
 'CATEGORY1_ID',
 'CATEGORY2',
 'CATEGORY2_ID',
 'CATEGORY3',
 'CATEGORY3_ID',
 'CATEGORY4',
 'CATEGORY4_ID',
 'CITY',
 'DATE_',
 'DISTRICT',
 'ITEMCODE',
 'ITEMID',
 'ITEMNAME',
 'LAT',
 'LON',
 'NAMESURNAME',
 'ORDERDETAILID',
 'ORDERID',
 'REGION',
 'STATUS_',
 'TOTALBASKET',
 'TOTALPRICE',
 'TOWN',
 'UNITPRICE',
 'USERBIRTHDATE',
 'USERGENDER',
 'USERID',
 'USERNAME_'}


In [9]:
frames["Orders.csv"]

In [46]:
item_details = (
        frames["Order_Details.csv"].join(
        frames["Categories_ENG.csv"],
        how="left",
        on="ITEMID",
    )
    .drop("ITEMID", "CATEGORY1_ID", "CATEGORY2_ID", "CATEGORY3_ID", "CATEGORY4_ID", "ITEMCODE_right")
)

In [47]:
item_desc = (
    item_details
    .select([
        "ITEMCODE",
        "CATEGORY1",
        "CATEGORY2",
        "CATEGORY3",
        "CATEGORY4",
        "BRAND",
        "UNITPRICE",
        "ITEMNAME"
    ])
    .unique("ITEMCODE")
)

In [48]:
order_details = (
    item_details
    .select([
        "ORDERID",
        "AMOUNT",
    ])
)

In [ ]:
total = (
        frames["Orders.csv"].join(
        frames["Customers_ENG.csv"],
        how="left",
        on="USERID",
    )
    .drop("USERID")
    .join(
        frames["Branches_ENG.csv"],
        how="left",
        on="BRANCH_ID",
    )
    .drop("BRANCH_ID")
    .join(
        order_details,
    #     (
    #         frames["Order_Details.csv"]
    #         .drop("ITEMCODE", "ORDERDETAILID")
            
    #     ),
        how="right",
        on="ORDERID",
    )
    # .head(10)
    .collect(engine="streaming")
)

In [ ]:
frames["Order_Details.csv"].select(pl.n_unique("ITEMID")).collect(engine="streaming")


len
u32
51185032


In [13]:
%time frames["Orders.csv"].select(pl.len()).collect(engine="streaming")
# %time frames["Order_Details.csv"].select(pl.len()).collect(engine="streaming")

CPU times: user 209 ms, sys: 150 ms, total: 360 ms
Wall time: 835 ms


len
u32
10235193


In [14]:
frames["Order_Details.csv"].join(frames["Orders.csv", how="left", on=""])select(pl.len()).collect(engine="streaming")

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (3403469151.py, line 1)